In [103]:
import numpy as np

In [104]:
T = 6
K = list('abcdefgh')
LINKS = [('a','c'), ('b','c'), ('c','d'), ('c','e'), 
         ('d','f'), ('d','g'), ('e','g'), ('e','h')]
CapacidadVeh = 20
CostoTransp = np.ones((len(K),len(K))) * 10

In [105]:
K2idx = {c: idx for idx, c in enumerate(K)}

In [106]:
Demanda0 = [0,0,0,30,20,40,40,40]
Inv0     = [100,80,10,0,0,0,0,0]
VehDisp0 = [10,10,0,0,0,0,0,0]

In [107]:
sum(Demanda0), sum(Inv0)

(170, 190)

In [108]:
Veh = np.zeros((len(K),len(K),T))
DemandaNS = np.zeros((len(K),T))
Consumo   = np.zeros((len(K),T))
Inv       = np.zeros((len(K),T))
VehDisp   = np.zeros((len(K),T))

In [109]:
Transp = np.zeros((len(K),len(K),T))

In [46]:
Transp[K2idx['a'],K2idx['c']] = [100,0,0,0,0,0]
Transp[K2idx['b'],K2idx['c']] = [ 80,0,0,0,0,0]
Transp[K2idx['c'],K2idx['d']] = [ 0,80,0,0,0,0]
Transp[K2idx['c'],K2idx['e']] = [0,100,0,0,0,0]
Transp[K2idx['d'],K2idx['f']] = [ 0,0,40,0,0,0]
Transp[K2idx['d'],K2idx['g']] = [  0,0,0,0,0,0]
Transp[K2idx['e'],K2idx['g']] = [ 0,0,40,0,0,0]
Transp[K2idx['e'],K2idx['h']] = [ 0,0,40,0,0,0]

In [110]:
Transp[K2idx['a'],K2idx['c']] = [ 80,0,0,0,0,0]
Transp[K2idx['b'],K2idx['c']] = [ 80,0,0,0,0,0]
Transp[K2idx['c'],K2idx['d']] = [10,100,0,0,0,0]
Transp[K2idx['c'],K2idx['e']] = [ 0,60,0,0,0,0]
Transp[K2idx['d'],K2idx['f']] = [ 0,0,40,0,0,0]
Transp[K2idx['d'],K2idx['g']] = [ 0,0,40,0,0,0]
Transp[K2idx['e'],K2idx['g']] = [ 0,0, 0,0,0,0]
Transp[K2idx['e'],K2idx['h']] = [ 0,0,40,0,0,0]

## Init

In [111]:
Inv[:,0]       = Inv0
VehDisp[:,0]   = VehDisp0
DemandaNS[:,0] = Demanda0

In [112]:
Inv

array([[100.,   0.,   0.,   0.,   0.,   0.],
       [ 80.,   0.,   0.,   0.,   0.,   0.],
       [ 10.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.]])

## Genotype => Phenotype

In [113]:
for t in range(1,T):
    for k in range(len(K)):
        consumo = max(0, min(Inv[k,t-1] \
                        + sum(Transp[j,k,t-1] for j in range(len(K))) \
                        - sum(Transp[k,j,t-1] for j in range(len(K))),
                            DemandaNS[k,t-1]))
        Inv[k,t] = Inv[k,t-1] \
                + sum(Transp[j,k,t-1] for j in range(len(K))) \
                - consumo \
                - sum(Transp[k,j,t-1] for j in range(len(K))) 
        DemandaNS[k,t] = DemandaNS[k,t-1] - consumo

In [114]:
Inv

array([[100.,  20.,  20.,  20.,  20.,  20.],
       [ 80.,   0.,   0.,   0.,   0.,   0.],
       [ 10., 160.,   0.,   0.,   0.,   0.],
       [  0.,   0.,  80.,   0.,   0.,   0.],
       [  0.,   0.,  40.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.]])

In [115]:
DemandaNS

array([[ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [30., 20.,  0.,  0.,  0.,  0.],
       [20., 20.,  0.,  0.,  0.,  0.],
       [40., 40., 40.,  0.,  0.,  0.],
       [40., 40., 40.,  0.,  0.,  0.],
       [40., 40., 40.,  0.,  0.,  0.]])

## Validation

In [116]:
import itertools
all(sum(Transp[k,j,t] for j in range(len(K))) <= Inv[k,t] \
    for k,t in itertools.product(range(len(K)), range(T)))

True

In [120]:
# Should be capacity check
for k,j,t in itertools.product(range(len(K)), range(len(K)), range(T)):
    Veh[k,j,t] = np.ceil(Transp[k,j,t] / CapacidadVeh) 

In [121]:
Veh

array([[[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [4., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [4., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [1., 5., 0., 0., 0., 0.],
        [0., 3., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        

In [75]:
for t in range(1,T):
    for k in range(len(K)):
        VehDisp[k,t] = VehDisp[k,t-1] \
                    + sum(Veh[j,k,t-1] for j in range(len(K))) \
                    - sum(Veh[k,j,t-1] for j in range(len(K)))

In [76]:
VehDisp

array([[10.,  5.,  5.,  5.,  5.,  5.],
       [10.,  6.,  6.,  6.,  6.,  6.],
       [ 0.,  9.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  4.,  2.,  2.,  2.],
       [ 0.,  0.,  5.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  2.,  2.,  2.],
       [ 0.,  0.,  0.,  2.,  2.,  2.],
       [ 0.,  0.,  0.,  2.,  2.,  2.]])